In [1]:
# installing H2O
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
  The repository located at h2o-release.s3.amazonaws.com is not a trusted or secure host and is being ignored. If this repository is available via HTTPS we recommend you use HTTPS instead, otherwise you may silence this warning and allow it anyway with '--trusted-host h2o-release.s3.amazonaws.com'.
    100% |████████████████████████████████| 120.9MB 158kB/s 
  Stored in directory: /root/.cache/pip/wheels/5e/0b/27/9eb72d708a3e7d1780ba296cf50b165d6d7a8ba6638915d7fe
Successfully built h2o


In [3]:
#importing and intializing

import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprcfo2bu7
  JVM stdout: /tmp/tmprcfo2bu7/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprcfo2bu7/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_unknownUser_m3yhlq
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
data_path = "https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/powerplant_output.csv"

In [8]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
df.describe()

Rows:9568
Cols:5




,TemperatureCelcius,ExhaustVacuumHg,AmbientPressureMillibar,RelativeHumidity,HourlyEnergyOutputMW
type,real,real,real,real,real
mins,1.81,25.36,992.89,25.56,420.26
mean,19.651231187290957,54.3058037207358,1013.2590781772578,73.30897784280936,454.36500940635455
maxs,37.11,81.56,1033.3,100.16,495.76
sigma,7.452473229611082,12.707892998326807,5.93878370581162,14.600268756728957,17.066994999803423
zeros,0,0,0,0,0
missing,0,0,0,0,0
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.4,1012.16,92.14,488.56


In [0]:
y = "HourlyEnergyOutputMW"
splits = df.split_frame(ratios=[0.8], seed=2019)
train = splits[0]
test = splits[1]

In [15]:
# 80% data used for building model
aml = H2OAutoML(max_runtime_secs=60, 
                seed=2019,
                project_name="powerplant_lb_frame")
aml.train(y=y, training_frame=train, leaderboard_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [16]:
# 100% data used used for building model
aml2 = H2OAutoML(max_runtime_secs=120,
                 seed=2019,
                 project_name="powerplant_full_data")
aml2.train(y=y, training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [17]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XRT_1_AutoML_20190211_053410,13.0317,3.60995,13.0317,2.49846,0.00791773
DRF_1_AutoML_20190211_053410,13.1184,3.62193,13.1184,2.49763,0.00794365
GLM_grid_1_AutoML_20190211_053410_model_1,22.0281,4.69341,22.0281,3.64924,0.0103351


In [18]:
aml2.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190211_053540,11.97,3.45977,11.97,2.52658,0.00760235
StackedEnsemble_BestOfFamily_AutoML_20190211_053540,11.97,3.45977,11.97,2.52658,0.00760235
XRT_1_AutoML_20190211_053540,12.0851,3.47637,12.0851,2.51562,0.00763504
DRF_1_AutoML_20190211_053540,12.1488,3.48552,12.1488,2.51488,0.0076597
GLM_grid_1_AutoML_20190211_053540_model_1,20.8071,4.56148,20.8071,3.6318,0.0100391


In [20]:
# prediction

pred = aml.predict(test)
pred.head()

drf prediction progress: |████████████████████████████████████████████████| 100%


predict
480.155
468.161
463.338
483.544
440.217
451.505
449.248
446.095
450.058
477.354


In [21]:
# performance
performance = aml.leader.model_performance(test)
performance


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 13.031739900661782
RMSE: 3.6099501244008594
MAE: 2.498459899151243
RMSLE: 0.007917729214267208
Mean Residual Deviance: 13.031739900661782
